In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from DecoderTrans import DecoderTransformer
from tokenizer import CurriculumTokenizer
from torch.nn.utils.rnn import pad_sequence
%matplotlib inline

Токены: [1, 13, 15, 5, 11, 14, 8, 12, 11, 2]
Обратное преобразование: <SOS>35-14=21<EOS>

Токены: [1, 11, 17, 6, 14, 15, 16, 8, 17, 17, 15, 12, 2]
Обратное преобразование: <SOS>17*456=7752<EOS>

Токены: [1, 11, 10, 12, 15, 3, 12, 8, 11, 2]
Обратное преобразование: <SOS>1025<UNK>2=1<EOS>
Токены: [1, 3, 11, 10, 12, 15, 3, 12, 3, 4, 13, 8, 11, 2]
Обратное преобразование: <SOS><UNK>1025<UNK>2<UNK>+3=1<EOS>


In [2]:
class MathDataset(torch.utils.data.Dataset):
    def __init__(self, expressions, targets):
        # expressions - это список/массив последовательностей, targets - это целевые значения
        self.expressions = expressions
        self.targets = targets
        self.max_length = max(len(exp) for exp in expressions)  # Определяем максимальную длину

    def __len__(self):
        return len(self.expressions)

    def __getitem__(self, idx):
        expression = self.expressions[idx]
        target = self.targets[idx]
        
        # Преобразуем выражение в список, если оно не в нужном формате
        if isinstance(expression, tuple):
            expression = list(expression)
        elif not isinstance(expression, list):
            raise ValueError(f"Expected list or tuple, got {type(expression)}")

        # Паддинг последовательности до максимальной длины
        padded_expression = self.pad_sequences(expression, self.max_length)
        
        return padded_expression, target
    
    @staticmethod
    def pad_sequences(seq, max_length):
        # Проверка на правильный тип данных
        if isinstance(seq, torch.Tensor):
            seq = seq.tolist()  # Преобразуем в список, если это тензор
        if not isinstance(seq, list):
            raise ValueError(f"Expected list or tensor, got {type(seq)}")

        padding_length = max_length - len(seq)
        
        # Добавляем паддинг (проверяем, что padding_length - это целое число)
        if padding_length > 0:
            padded_seq = F.pad(torch.tensor(seq, dtype=torch.long), (0, padding_length), value=0)
        else:
            padded_seq = torch.tensor(seq, dtype=torch.long)
        
        return padded_seq

seq = [1, 2, 3, 4]
max_length = 10

padded_seq = MathDataset.pad_sequences(seq, max_length)
print(padded_seq)

tensor([1, 2, 3, 4, 0, 0, 0, 0, 0, 0])


In [3]:
def collate_fn(batch):
    expressions, targets = zip(*batch)
    max_length = max(len(exp) for exp in expressions)
    expressions_padded = [MathDataset.pad_sequences(exp, max_length) for exp in expressions]
    return torch.stack(expressions_padded), torch.tensor(targets)
def generate_mask(tensor, pad_value=0):
    return (tensor != pad_value).float()


In [4]:
def safe_tokenize(expression, tokenizer, vocab_size):
    """
    Безопасная токенизация: если токен выходит за пределы словаря, он заменяется на unk_token.
    """
    tokens = tokenizer.encode(expression)  # Используем метод encode, а не tokenize
    return [
        token if token < vocab_size else tokenizer.vocab['<UNK>']  # Подставляем <UNK> для недопустимых индексов
        for token in tokens
    ]

In [5]:
# Загружаем данные из CSV
train_data = pd.read_csv('shuffled_data.csv')
test_data = pd.read_csv('test_data.csv')

# Предполагается, что у вас есть ваш токенизатор и спецификации для токенов
tokenizer = CurriculumTokenizer()
vocab = tokenizer.vocab  # Получаем словарь токенизатора
vocab_size = len(vocab)  # Определяем размер словаря

# Токенизация данных с использованием исправленной безопасной токенизации
train_tokens = [safe_tokenize(expr, tokenizer, vocab_size) for expr in train_data['expression']]
test_tokens = [safe_tokenize(expr, tokenizer, vocab_size) for expr in test_data['expression']]

# Создаем датасеты
train_dataset = MathDataset(train_tokens, train_data['result'].values)
test_dataset = MathDataset(test_tokens, test_data['result'].values)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DecoderTransformer(num_tokens=tokenizer.vocab_size, n_embd=128).to(device)  # Исправлено здесь
criterion = nn.MSELoss()  # Поскольку мы предсказываем числа
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def create_targets(input_sequence):
    targets = input_sequence[1:].clone()  # Сдвигаем на 1
    padding = input_sequence.new_zeros((1, input_sequence.size(1)))  # Добавляем паддинг
    targets = torch.cat([padding, targets], dim=0)
    
    # Проверка индексов
    assert torch.max(targets) < vocab_size, f"Target index {torch.max(targets)} out of range!"
    return targets


In [8]:
def train_model(model, data_loader, criterion, optimizer, device, vocab_size):
    model.train()
    total_loss = 0.0
    for batch in data_loader:
        expressions_tensor, token_ids = batch

        # Проверка максимального и минимального индекса
        print(f"Max token index in batch: {token_ids.max()}")
        print(f"Min token index in batch: {token_ids.min()}")

        # Проверка индексов, выходящих за пределы словаря
        if token_ids.max() >= vocab_size or token_ids.min() < 0:
            raise ValueError(f"Token index out of range: max {token_ids.max()}, vocab_size {vocab_size}")
        
        # Приведение токенов к допустимым индексам (если нужно)
        token_ids = token_ids.clamp(0, vocab_size - 1)

        expressions_tensor, token_ids = expressions_tensor.to(device), token_ids.to(device)

        optimizer.zero_grad()

        # Прямой проход (model принимает два аргумента)
        outputs = model(expressions_tensor, token_ids)  # Передаем два аргумента
        
        # Потеря
        loss = criterion(outputs.view(-1, vocab_size), token_ids.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)


def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for expressions, targets in data_loader:
            expressions = expressions.long().to(device)  # Convert to LongTensor
            targets = targets.to(device).float()  # Ensure targets are float32

            # Проверка размерности перед передачей в модель
            print(f'Input expressions tensor shape: {expressions.shape}')
            print(f'Input targets tensor shape: {targets.shape}')
            
            # Выводим максимальные индексы
            print(f'Max index in expressions_tensor: {expressions.max()}')
            
            outputs = model(expressions)

            # Вычисление потери
            loss = criterion(outputs.squeeze(), targets)
            total_loss += loss.item()

    return total_loss / len(data_loader)


In [9]:
vocab_size = tokenizer.vocab_size
num_epochs = 10
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer, device, vocab_size)
    test_loss = evaluate_model(model, test_loader, device)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Max token index in batch: 8366
Min token index in batch: 1


ValueError: Token index out of range: max 8366, vocab_size 19

In [ ]:
# Визуализация потерь
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()
plt.show()